# テーマC：量子化の前処理としてのアダマール変換の評価
[Open In Colab](https://colab.research.google.com/github/ArtIC-TITECH/b3-proj-2025/blob/main/theme_C/theme_C.ipynb)

## モジュールの読み込み

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.autograd import Function

## MNISTのデータセット/精度評価関数の作成

In [15]:
# 実行デバイスの設定
device = 'cuda:2'

# 普通のtransform
transform_normal = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# テストデータには普通のtransformを使ってください
transform_for_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform_normal) # モデルの学習に使うデータセット
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform_for_test) # モデルの評価に使うデータセット
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

def compute_accuracy(model, test_loader, device='cuda:0'):
    model.eval()  # 評価モード
    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images.to(device))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to(device)).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

def train(model, lr=0.05, epochs=5, device='cuda:0'):
    # 損失関数と最適化手法の定義
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(epochs):
        loss_sum = 0
        for images, labels in train_loader:
            # モデルの予測
            outputs = model(images.to(device))

            # 損失の計算
            loss = criterion(outputs, labels.to(device))
            loss_sum += loss.item()

            # 勾配の初期化
            optimizer.zero_grad()

            # バックプロパゲーション
            loss.backward()

            # オプティマイザの更新
            optimizer.step()

        # 損失を表示
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss_sum/len(train_loader):.4f}')
    return model




## 通常モデルの学習

In [25]:
class SimpleModel(nn.Module):
    def __init__(self): # モデルのセットアップ
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x): # モデルが行う処理
        x = x.view(-1, 28 * 28)  # 28x28の画像を１次元に変換
        x = self.fc1(x) 
        x = nn.ReLU()(x) # 活性化関数
        x = self.fc2(x) 
        return x

# モデルのインスタンスを作成
model = SimpleModel().to(device)

model = train(model, lr=0.1, epochs=10, device=device)

Epoch [1/10], Loss: 0.4222
Epoch [2/10], Loss: 0.1989
Epoch [3/10], Loss: 0.1546
Epoch [4/10], Loss: 0.1305
Epoch [5/10], Loss: 0.1130
Epoch [6/10], Loss: 0.1019
Epoch [7/10], Loss: 0.0921
Epoch [8/10], Loss: 0.0854
Epoch [9/10], Loss: 0.0808
Epoch [10/10], Loss: 0.0727


精度の確認

In [26]:
accuracy = compute_accuracy(model, test_loader, device=device)

Accuracy: 96.83%


## スカラー量子化（一様対称量子化）の実行

###  プロセス：量子化層に変換-->量子化認識学習

ここでは簡便に量子化パラメータをmin-maxスケーリングで決定する
対称量子化なので、行列Xの最大値と最小値の差の２分の1を$p$-bitの数値範囲の最大値$q_{max}$でわる

$q_{max} = 2^{(p-1)} - 1$

$s = \frac{max(X) - min(X)}{2q_{max}}$

$X_{q} = s * \text{clip}(\text{round}(\frac{X}{s}), -q_{max}, q_{max})$

In [20]:

class SymQuantSTE(Function):
    @staticmethod
    def forward(ctx, input: torch.Tensor, scale: torch.Tensor, num_bits: int):
        if num_bits == 1:
            s = scale.abs()
            output = s * torch.sgn(input)
        else:
            s = scale.abs().clamp_min(1e-8)
            qmax = 2 ** (num_bits - 1) - 1
            q = torch.clamp(torch.round(input / s), -qmax, qmax)
            output = q * s

        # backward用に保存
        ctx.save_for_backward(input, s)
        ctx.num_bits = num_bits
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, s = ctx.saved_tensors   # forwardでsaveしたものを正しく取り出す
        num_bits = ctx.num_bits
        if num_bits == 1:
            grad_input = torch.clamp(grad_output, -1, 1)
        else:
            qmax = 2 ** (num_bits - 1) - 1
            mask = (input.abs() <= qmax * s).to(grad_output.dtype)
            grad_input = grad_output * mask

        return grad_input, None, None




class SymQuantLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True, weight_bits=8, act_bits=None):
        super().__init__(in_features, out_features, bias)
        self.weight_bits = weight_bits
        self.act_bits = act_bits

    def forward(self, input):
        # weight のスケール
        if self.weight_bits == 1:
            weight_scale = self.weight.abs().sum() / self.weight.numel()
        else:
            qmax_w = 2 ** (self.weight_bits - 1) - 1
            weight_scale = (self.weight.max() - self.weight.min()) / (2 * qmax_w)

        # activation のスケール
        if self.act_bits is not None:
            if self.act_bits == 1:
                act_scale = input.abs().sum() / input.numel()
            else:
                qmax_a = 2 ** (self.act_bits - 1) - 1
                act_scale = (input.max() - input.min()) / (2 * qmax_a)
            input = SymQuantSTE.apply(input, act_scale, self.act_bits)

        # quantized weight
        w_q = SymQuantSTE.apply(self.weight, weight_scale, self.weight_bits)

        return F.linear(input, w_q, self.bias)



def replace_linear_with_quantizedlinear(module, weight_bits=8, act_bits=None):
    for name, child in module.named_children():
        # すでに QuantizedLinear ならスキップ
        if isinstance(child, SymQuantLinear):
            continue
        if isinstance(child, nn.Linear):
            qlinear = SymQuantLinear(
                child.in_features,
                child.out_features,
                bias=(child.bias is not None),
                weight_bits=weight_bits,
                act_bits=act_bits
            )
            # 重みとバイアスをコピー
            qlinear.weight.data.copy_(child.weight.data)
            if child.bias is not None:
                qlinear.bias.data.copy_(child.bias.data)
            setattr(module, name, qlinear)
        else:
            replace_linear_with_quantizedlinear(child, weight_bits, act_bits)
    return module

# モデルのインスタンスを作成
model = SimpleModel().to(device)
# 通常学習
print('warming up by no-quantized training...')
model = train(model, lr=0.1, epochs=5, device=device)
# Linear層をQuantizedLinearに置換
model_q = replace_linear_with_quantizedlinear(model, weight_bits=1, act_bits=1)
print('quantization aware training...')
model_q = train(model_q, lr=1e-2, epochs=10, device=device)
accuracy = compute_accuracy(model_q, test_loader)

warming up by no-quantized training...
Epoch [1/5], Loss: 0.4251
Epoch [2/5], Loss: 0.2056
Epoch [3/5], Loss: 0.1570
Epoch [4/5], Loss: 0.1307
Epoch [5/5], Loss: 0.1166
quantization aware training...
Epoch [1/10], Loss: 1.5707
Epoch [2/10], Loss: 0.9505
Epoch [3/10], Loss: 0.7470
Epoch [4/10], Loss: 0.6746
Epoch [5/10], Loss: 0.6514
Epoch [6/10], Loss: 0.6492
Epoch [7/10], Loss: 0.5975
Epoch [8/10], Loss: 0.5278
Epoch [9/10], Loss: 0.5058
Epoch [10/10], Loss: 0.5199
Accuracy: 83.57%


## アダマール変換付きスカラー量子化（一様対称量子化）の実行

###  プロセス：アダマール変換付き量子化層に変換-->量子化認識学習

In [22]:



class SymQuantSTE(Function):
    @staticmethod
    def forward(ctx, input: torch.Tensor, scale: torch.Tensor, num_bits: int):
        if num_bits == 1:
            s = scale.abs()
            output = s * torch.sgn(input)
        else:
            s = scale.abs().clamp_min(1e-8)
            qmax = 2 ** (num_bits - 1) - 1
            q = torch.clamp(torch.round(input / s), -qmax, qmax)
            output = q * s

        ctx.save_for_backward(input, s)
        ctx.num_bits = num_bits
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, s = ctx.saved_tensors
        num_bits = ctx.num_bits
        if num_bits == 1:
            mask = (input.abs() <= s).to(grad_output.dtype)
            grad_input = torch.clamp(grad_output, -1, 1)
        else:
            qmax = 2 ** (num_bits - 1) - 1
            mask = (input.abs() <= qmax * s).to(grad_output.dtype)
            grad_input = grad_output * mask

        return grad_input, None, None


def hadamard_matrix(n: int, device=None, dtype=None) -> torch.Tensor:
    """
    n次のアダマール行列を生成 (nは2のべき乗)
    """
    if n == 1:
        return torch.tensor([[1.0]], device=device, dtype=dtype)
    H = hadamard_matrix(n // 2, device=device, dtype=dtype)
    top = torch.cat([H, H], dim=1)
    bottom = torch.cat([H, -H], dim=1)
    return torch.cat([top, bottom], dim=0)


def clipped_hadamard(n: int, device=None, dtype=None) -> torch.Tensor:
    """
    n次元入力に対応するクリップ版アダマール行列を返す
    - 入力次元nに対して、2^k >= n を探す
    - 2^k 次のアダマール行列を作り、左上の n×n を取り出す
    """
    k = (n - 1).bit_length()  # ceil(log2(n))
    H_full = hadamard_matrix(2**k, device=device, dtype=dtype)
    H_clip = H_full[:n, :n]
    return H_clip / (n**0.5)  # 正規化


class HadamardSymQuantLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True, weight_bits=8, act_bits=None):
        super().__init__(in_features, out_features, bias)
        self.weight_bits = weight_bits
        self.act_bits = act_bits
        # 固定Hadamard行列をbufferに登録（学習しない）
        H = clipped_hadamard(in_features)
        self.register_buffer("H", H)

    def forward(self, input):
        # --- アダマール変換 ---
        input = input @ self.H.T   # (batch, in_features) × (in_features, in_features)

        # --- weight のスケール ---
        if self.weight_bits is not None:
            if self.weight_bits == 1:
                weight_scale = self.weight.abs().sum() / self.weight.numel()
            else:
                qmax_w = 2 ** (self.weight_bits - 1) - 1
                weight_scale = (self.weight.max() - self.weight.min()) / (2 * qmax_w)
            # --- quantized weight ---
            w_q = SymQuantSTE.apply(self.weight, weight_scale, self.weight_bits)
        else:
            weight_scale = None
            w_q = self.weight

        # --- activation のスケール ---
        if self.act_bits is not None:
            if self.act_bits == 1:
                act_scale = input.abs().sum() / input.numel()
            else:
                qmax_a = 2 ** (self.act_bits - 1) - 1
                act_scale = (input.max() - input.min()) / (2 * qmax_a)
            input = SymQuantSTE.apply(input, act_scale, self.act_bits)

        return F.linear(input, w_q, self.bias)




def replace_linear_with_hadamard_quantizedlinear(module, weight_bits=8, act_bits=None):
    for name, child in module.named_children():
        # すでに QuantizedLinear ならスキップ
        if isinstance(child, HadamardSymQuantLinear):
            qlinear = HadamardSymQuantLinear(
                child.in_features,
                child.out_features,
                bias=(child.bias is not None),
                weight_bits=weight_bits,
                act_bits=act_bits
            )
            # 重みとバイアスをコピー
            qlinear.weight.data.copy_(child.weight.data)
            if child.bias is not None:
                qlinear.bias.data.copy_(child.bias.data)
            setattr(module, name, qlinear)
        if isinstance(child, nn.Linear):
            qlinear = HadamardSymQuantLinear(
                child.in_features,
                child.out_features,
                bias=(child.bias is not None),
                weight_bits=weight_bits,
                act_bits=act_bits
            )
            # 重みとバイアスをコピー
            qlinear.weight.data.copy_(child.weight.data)
            if child.bias is not None:
                qlinear.bias.data.copy_(child.bias.data)
            setattr(module, name, qlinear)
        else:
            replace_linear_with_hadamard_quantizedlinear(child, weight_bits, act_bits)
    return module

# モデルのインスタンスを作成
model = SimpleModel().to(device)

# 通常学習
model = replace_linear_with_hadamard_quantizedlinear(model, weight_bits=None, act_bits=None)
print('warming up by no-quantized training...')
model = train(model, lr=0.1, epochs=5, device=device)
# Linear層をHadamardQuantizedLinearに置換
model_q = replace_linear_with_hadamard_quantizedlinear(model, weight_bits=1, act_bits=1)
print('hadamard transformation and quantization aware training...')
model_q = train(model_q, lr=1e-2, epochs=10, device=device)
accuracy = compute_accuracy(model_q, test_loader)

warming up by no-quantized training...
Epoch [1/5], Loss: 0.5063
Epoch [2/5], Loss: 0.2324
Epoch [3/5], Loss: 0.1804
Epoch [4/5], Loss: 0.1535
Epoch [5/5], Loss: 0.1358
hadamard transformation and quantization aware training...
Epoch [1/10], Loss: 1.1305
Epoch [2/10], Loss: 0.6893
Epoch [3/10], Loss: 0.5900
Epoch [4/10], Loss: 0.5412
Epoch [5/10], Loss: 0.5137
Epoch [6/10], Loss: 0.4973
Epoch [7/10], Loss: 0.4788
Epoch [8/10], Loss: 0.4680
Epoch [9/10], Loss: 0.4615
Epoch [10/10], Loss: 0.4535
Accuracy: 85.43%


In [21]:
class RandomizedHadamardSymQuantLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True, weight_bits=8, act_bits=None):
        super().__init__(in_features, out_features, bias)
        self.weight_bits = weight_bits
        self.act_bits = act_bits
        
        # Hadamard 行列
        H = clipped_hadamard(in_features)
        # ランダム ±1 符号
        D = torch.randint(0, 2, (in_features,), dtype=torch.float32) * 2 - 1
        H_rht = H * D.unsqueeze(0)  # 列ごとに符号を掛ける
        self.register_buffer("H_rht", H_rht)

    def forward(self, input):
        # --- Randomized Hadamard Transform ---
        input = input @ self.H_rht.T

        # --- weight のスケール ---
        if self.weight_bits is not None:
            if self.weight_bits == 1:
                weight_scale = self.weight.abs().sum() / self.weight.numel()
            else:
                qmax_w = 2 ** (self.weight_bits - 1) - 1
                weight_scale = (self.weight.max() - self.weight.min()) / (2 * qmax_w)
            # --- quantized weight ---
            w_q = SymQuantSTE.apply(self.weight, weight_scale, self.weight_bits)
        else:
            w_q = self.weight

        # --- activation のスケール ---
        if self.act_bits is not None:
            if self.act_bits == 1:
                act_scale = input.abs().sum() / input.numel()
            else:
                qmax_a = 2 ** (self.act_bits - 1) - 1
                act_scale = (input.max() - input.min()) / (2 * qmax_a)
            input = SymQuantSTE.apply(input, act_scale, self.act_bits)

        return F.linear(input, w_q, self.bias)
    

def replace_linear_with_randomized_hadamard_quantizedlinear(module, weight_bits=8, act_bits=None):
    for name, child in module.named_children():
        if isinstance(child, RandomizedHadamardSymQuantLinear):
            qlinear = RandomizedHadamardSymQuantLinear(
                child.in_features,
                child.out_features,
                bias=(child.bias is not None),
                weight_bits=weight_bits,
                act_bits=act_bits
            )
            # 重みとバイアスをコピー
            qlinear.weight.data.copy_(child.weight.data)
            if child.bias is not None:
                qlinear.bias.data.copy_(child.bias.data)
            setattr(module, name, qlinear)
        if isinstance(child, nn.Linear):
            qlinear = RandomizedHadamardSymQuantLinear(
                child.in_features,
                child.out_features,
                bias=(child.bias is not None),
                weight_bits=weight_bits,
                act_bits=act_bits
            )
            # 重みとバイアスをコピー
            qlinear.weight.data.copy_(child.weight.data)
            if child.bias is not None:
                qlinear.bias.data.copy_(child.bias.data)
            setattr(module, name, qlinear)
        else:
            replace_linear_with_randomized_hadamard_quantizedlinear(child, weight_bits, act_bits)
    return module


# モデルのインスタンスを作成
model = SimpleModel().to(device)
# 通常学習
print('warming up by no-quantized training...')
model = replace_linear_with_randomized_hadamard_quantizedlinear(model, weight_bits=None, act_bits=None)
model = train(model, lr=0.1, epochs=5, device=device)
# Linear層をHadamardQuantizedLinearに置換
model_q = replace_linear_with_randomized_hadamard_quantizedlinear(model, weight_bits=1, act_bits=1)
print('randomized hadamard transformation and quantization aware training...')
model_q = train(model_q, lr=1e-2, epochs=10, device=device)
accuracy = compute_accuracy(model_q, test_loader)

warming up by no-quantized training...
Epoch [1/5], Loss: 0.4199
Epoch [2/5], Loss: 0.1933
Epoch [3/5], Loss: 0.1486
Epoch [4/5], Loss: 0.1263
Epoch [5/5], Loss: 0.1115
randomized hadamard transformation and quantization aware training...
Epoch [1/10], Loss: 1.0163
Epoch [2/10], Loss: 0.6615
Epoch [3/10], Loss: 0.5934
Epoch [4/10], Loss: 0.5739
Epoch [5/10], Loss: 0.5602
Epoch [6/10], Loss: 0.5377
Epoch [7/10], Loss: 0.5141
Epoch [8/10], Loss: 0.5040
Epoch [9/10], Loss: 0.4932
Epoch [10/10], Loss: 0.4918
Accuracy: 85.01%


## モデルサイズの確認

In [ ]:
def compute_model_size(model: nn.Module):
    total_size = 0
    for module in model.modules():
        if isinstance(module, nn.Linear):
            # 通常の Linear 層のサイズ (float32)
            total_size += module.weight.numel() * 4  # float32 として計算
            if module.bias is not None:
                total_size += module.bias.numel() * 4  # float32 として計算
        elif isinstance(module, (SymQuantLinear, HadamardSymQuantLinear, RandomizedHadamardSymQuantLinear)):
            if module.weight_bits is None:
                module.weight_bits = 32  # float32として計算
            # weight のサイズ
            total_size += module.weight.numel() * module.weight_bits/8  
            # bias のサイズ
            if module.bias is not None:
                total_size += module.bias.numel() * 4  # float32 として計算
    return total_size / (1024 * 1024)  # MB単位で返す

model_size = compute_model_size(model_q)
print(f'Model size: {model_size:.2f} MB')

## 課題
### ・普通に量子化した場合、アダマール変換を入れて量子化した場合、ランダマイズドアダマール変換を入れて量子化した場合の精度をそれぞれ比較する（低ビットで比較しないとわからないかも）
### ・実験結果について考察する